In [1]:
import pandas as pd
import numpy as np

## prépare un mapping de l'index des circonscriptions de 2024 pour l'aligner avec celui de 2022

In [3]:
mapping_dept = (
    pd
    .read_csv('lg2022_circonscriptions.csv')
    .groupby(['CodDpt3Car', 'CodDpt'])
    .agg({'CodCirLg': len})
    .reset_index()
    .set_index('CodDpt3Car')
    [['CodDpt']]
    .query('CodDpt.str.startswith("Z")')
    .to_dict()
    ['CodDpt']
)

mapping_dept

{'099': 'ZZ',
 '971': 'ZA',
 '972': 'ZB',
 '973': 'ZC',
 '974': 'ZD',
 '975': 'ZS',
 '976': 'ZM',
 '977': 'ZX',
 '986': 'ZW',
 '987': 'ZP',
 '988': 'ZN'}

In [4]:
# candidats_dgfr = (
#     pd
#     .read_csv('https://www.data.gouv.fr/fr/datasets/r/9efe7b76-8257-4db5-9e9f-37abb81ce65d')
#     .assign(
#         NumCirc = lambda df: df['Code circonscription'].str[-2:].str.pad(3, fillchar="0"),
#         Dept = lambda df: df['Code circonscription'].str[:-2].str.pad(2, fillchar="0").str.pad(2, fillchar="0"),
#         CodCirc = lambda df: df.Dept.replace(mapping_dept) + df.NumCirc
#     )
# )

# candidats_dgfr

## récupère la liste des circonscriptions

In [2]:
territoires_url = 'https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2024/territoires/territoires.xml'

In [19]:
circonscriptions = (
    pd
    .read_xml(
        territoires_url,
        xpath='.//Circonscription'
    )
    .assign(
        CodDpt = lambda df: df.CodCirElec.str[:-2]
    )
    .set_index('CodCirElec')
    .drop(['Communes'], axis=1)
)

circonscriptions

,LibCirElec,NbSap,CodDpt
CodCirElec,,,
97501,Saint-Pierre-et-Miquelon,1,975
98601,1ère circonscription,1,986
98701,1ere circonscription,1,987
98702,2ème circonscription,1,987
98703,3ème circonscription,1,987
...,...,...,...
8405,5ème circonscription,1,84
2A01,1ère circonscription,1,2A
2A02,2ème circonscription,1,2A


In [20]:
circonscriptions.to_csv('lg2024_circonscriptions.csv')

## récupère la liste des candidat·e·s du premier tour

In [6]:
def get_candidats_t1(circonscription):
    dept = circonscription[:-2]
    circ = circonscription

    url = f'https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2024/candidatsT1/{dept}/C1{dept}{circonscription}.xml'
        
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath='.//Candidat'
            )
            .assign(CodCirc=circonscription)
        )
    except:
        print(f'error: {circonscription}')
        df = None
        
    
    return df
    
    
candidats_2024_t1 = pd.concat(
    [
        get_candidats_t1(circonscription)
        for circonscription in circonscriptions.index.to_list()
    ]
)

candidats_2024_t1

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,CodCirc
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,97501
1,2,BEAUMONT,Frédéric,M.,SOC,Parti socialiste,97501
2,3,CHAGNON,Patricia,Mme,RN,Rassemblement National,97501
3,4,LETOURNEL,Marion,Mme,FI,La France insoumise,97501
4,5,LEBAILLY,Patrick,M.,DVG,Divers gauche,97501
...,...,...,...,...,...,...,...
3,4,RONGIONE,Viviane,Mme,EXG,Extrême gauche,2B02
4,5,GIACOMI,Jean-Antoine,M.,REG,Régionaliste,2B02
5,6,JOUART,Sylvie,Mme,RN,Rassemblement National,2B02
6,7,CARLI,Antò,M.,REG,Régionaliste,2B02


In [7]:
candidats_2024_t1.to_csv('lg2024_t1_candidats.csv', index=False)

### transpose la liste des candidat·e·s dans un format *wide*

In [10]:
candidats_2024_t1_wide = (
    candidats_2024_t1
    .assign(
        Nom_Prenom = lambda df: df.NomPsn + " " +df.PrenomPsn
    )
    .pivot_table(
        index = 'CodCirc',
        columns = 'CodNuaCand',
        values = 'Nom_Prenom',
        aggfunc= lambda x : ', '.join(x)
    )
    .replace(np.nan, '')
)

candidats_2024_t1_wide

CodNuaCand,COM,DIV,DSV,DVC,DVD,DVG,ECO,ENS,EXD,EXG,...,LR,RDG,REC,REG,RN,SOC,UDI,UG,UXD,VEC
CodCirc,,,,,,,,,,,,,,,,,,,,,
0101,,,"VINCENT Cyril, MENDES Michael",,,,,GUILLERMIN Vincent,,LAHY Éric,...,BRETON Xavier,,,,MAÎTRE Christophe,,,GUERAUD Sébastien,,
0102,,,,DAUBIÉ Romain,,,,,EYRAUD Olivier,GOUTAGNY Vincent,...,NANCHI Alexandre,,,,KOTARAC Andréa,,,MEYER Maxime,,
0103,,"TONIZZO Sofia, KOUASSI Fulgence",VEILLEROT Annick,,,,,GIVERNET Olga,,MAISONNETTE Cécile,...,UNAL Khadija,,,,DUBARRY Karine,,,JOLIE Christian,,
0104,,"BRESSON Yannick, NICAUD Jérémy",,,,,,COQUELET Christophe,,COUSSON Sylvain,...,BILLOUDET Guy,,,,BUISSON Jérôme,,,LIOTIER Charline,,
0105,,,,,ABAD Damien,,CHATELARD Thomas,DESCOURS Nathalie,,CROZET Sylvie,...,BOURDIN Fabrice,,PATRU Maria Cristina,,,,,PISANI Florence,CHAVENT Marc,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ07,,"GEFFRAY Fanny, CHAMBON Jérôme",,HUQUET Isabelle,,RICHARD Cécile,,PETIT Frédéric,,,...,MIER-GARRIGOU Dominique,,ALEXANDRE Julie,,NAVEYS--DUMAS Mathilde,,,RHARMAOUI-CLAQUIN Asma,,
ZZ08,,"SPITALAS Nicolas, BIZET David, SIGOURA Benjamin",,NEFFATI Gilles,HABABOU SOLOMON Philippe,CHARTRAIN Valérie,,YADAN Caroline,,,...,"HABIB Meyer, ASSOULINE Aurelie",,BENSOUSSAN Guillaume,,,,,LERER Yaël,,
ZZ09,,"OUDRHIRI Hassan, SIDIBÉ Gabriel Marie, DIANIFA...",FADILI Hachim,"TAHIRI Rachid, DUCELLIER Régina","SACKHO Kourtoum, DAVOUX Erwan Borhan","BOUDJEKADA Ismaël, TINAUGUS Edouard, KHALFI Se...",,DJOUADI Samira,,,...,BADREDDINE Jihad,,DREVON Pierre,,CHARRON Elodie,,,BEN CHEÏKH Karim,,


In [11]:
candidats_2024_t1_wide.to_csv('lg2024_t1_candidats.wide.csv')

## précalcule un tableau sur la présence des nuances dans chacune des circonscriptions

In [8]:
nuances2024 = (
    candidats_2024_t1
    .pivot_table(
        index = 'CodCirc',
        columns = 'CodNuaCand',
        values = 'NomPsn',
        aggfunc= lambda x : len(x) >= 1
    )
    .fillna(False)
)

nuances2024

CodNuaCand,COM,DIV,DSV,DVC,DVD,DVG,ECO,ENS,EXD,EXG,...,LR,RDG,REC,REG,RN,SOC,UDI,UG,UXD,VEC
CodCirc,,,,,,,,,,,,,,,,,,,,,
0101,False,False,True,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
0102,False,False,False,True,False,False,False,False,True,True,...,True,False,False,False,True,False,False,True,False,False
0103,False,True,True,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
0104,False,True,False,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
0105,False,False,False,False,True,False,True,True,False,True,...,True,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ07,False,True,False,True,False,True,False,True,False,False,...,True,False,True,False,True,False,False,True,False,False
ZZ08,False,True,False,True,True,True,False,True,False,False,...,True,False,True,False,False,False,False,True,False,False
ZZ09,False,True,True,True,True,True,False,True,False,False,...,True,False,True,False,True,False,False,True,False,False


In [9]:
nuances2024.to_csv('lg2024_t1_nuances.csv')